In [ ]:
!pip install tensorflow==1.13.2 opencv-python slim slidingwindow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.7 MB 34 kB/s 
     |████████████████████████████████| 176 kB 60.7 MB/s 
     |████████████████████████████████| 3.2 MB 52.5 MB/s 
     |████████████████████████████████| 367 kB 35.1 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 149 kB 69.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 256 kB 64.5 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 219 kB 67.7 MB/s 
     |████████████████████████████████| 3.8 MB 36.6 MB/s 
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=7e4f1b3b2c3b343a83ba6cf2fcc1502b35b75700a2ace86e3f1e5

In [ ]:
!apt install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,764 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpa

In [ ]:
!git clone https://github.com/dronefreak/human-action-classification.git && mv human-action-classification/* . && rm -drf human-action-classification


Cloning into 'human-action-classification'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 339 (delta 0), reused 0 (delta 0), pack-reused 335
Receiving objects: 100% (339/339), 102.03 MiB | 31.05 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [ ]:
!cd tf_pose/pafprocess/ && swig -python -c++ pafprocess.i && python setup.py build_ext --inplace

running build_ext
building '_pafprocess' extension
swigging pafprocess.i to pafprocess_wrap.cpp
swig -python -c++ -o pafprocess_wrap.cpp pafprocess.i
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I. -I/usr/include/python3.7m -c pafprocess.cpp -o build/temp.linux-x86_64-3.7/pafprocess.o
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I. -I/usr/include/python3.7m -c pafprocess_wrap.cpp -o build/t

In [ ]:
from google.colab import drive

#mount drive
mount_point= '/content/gdrive'
drive.mount(mount_point)

Mounted at /content/gdrive


In [ ]:
import argparse
import logging
import time
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import scripts.label_image as label_img
import scripts.label_image_scene as label_img_scene

if __name__ == '__main__':
 
    IMAGE = '/content/gdrive/MyDrive/Colab Notebooks/AIA22_STUDIO/220525_SplitScenes/01/scene3.png'

    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
    image = cv2.imread(IMAGE)

	# count = 0
	
    start_time = time.time()
    humans = e.inference(image, upsample_size=4.0)
    img = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
	
    # Getting only the skeletal structure (with white background) of the actual image
    image = np.zeros(image.shape,dtype=np.uint8)
    image.fill(255) 
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
	
    # Classification
    pose_class = label_img.classify(image)
    scene_class = label_img_scene.classify(IMAGE)
    end_time = time.time()
    cv2.putText(img,
				"Predicted Pose: %s" %(pose_class),
				(10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
				(0, 0, 255), 2)
    cv2.putText(img,
				"Predicted Scene: %s" %(scene_class),
				(10, 30),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
				(0, 0, 255), 2)
    print('\n Overall Evaluation time (1-image): {:.3f}s\n'.format(end_time-start_time))
    cv2.imwrite(f'show_{IMAGE}',img)
    cv2_imshow(img)

#**MODIFIED OPEN-POSE**

It runs over the images in a folder and appends pose and activity to lists for further analysis.

In [ ]:
import argparse
import logging
import time
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import scripts.label_image as label_img
import scripts.label_image_scene as label_img_scene

In [ ]:
path = '/content/gdrive/MyDrive/Colab Notebooks/YOLO_testset/OpenPose_testset/'

# Just needed in case you'd like to append it to an array
img_paths = []

# for filename in os.listdir(path):
#     if filename.endswith("jpg" or "png"): 
#         img_paths.append(path + filename)

# Some names might be weird and not append properly, when this happens the following way works
# Only import this way when you know all the files in the folder are images
for filename in os.listdir(path):
  img_paths.append(path + filename)

In [ ]:
img_paths

['/content/gdrive/MyDrive/Colab Notebooks/YOLO_testset/OpenPose_testset/Screenshot 2022-05-12 at 16.08.47.png',
 '/content/gdrive/MyDrive/Colab Notebooks/YOLO_testset/OpenPose_testset/Screenshot_Trim1.png',
 '/content/gdrive/MyDrive/Colab Notebooks/YOLO_testset/OpenPose_testset/Screenshot_Trim2.png',
 '/content/gdrive/MyDrive/Colab Notebooks/YOLO_testset/OpenPose_testset/Screenshot_Trim3.png']

In [ ]:
# img_paths

In [ ]:
poses = []
activities = []

if __name__ == '__main__':

  for p in img_paths:
    # OpenPose estimation modified by pablo, if not working return to source code
    IMAGE = p

    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
    image = cv2.imread(IMAGE)

    humans = e.inference(image, upsample_size=4.0)
    img = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

    # Getting only the skeletal structure (with white background) of the actual image
    image = np.zeros(image.shape,dtype=np.uint8)
    image.fill(255) 
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

    # Classification
    poses.append(label_img.classify(image))
    activities.append(label_img_scene.classify(IMAGE))

    #Plotting image and prediction, comment section to save time
    cv2.putText(img,
				"Predicted Pose: %s" %(label_img.classify(image)),
				(10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
				(0, 0, 255), 2)
    cv2.putText(img,
				"Predicted Scene: %s" %(label_img_scene.classify(IMAGE)),
				(10, 30),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
				(0, 0, 255), 2)
    cv2.imwrite(f'show_{IMAGE}',img)
    cv2_imshow(img)

**EVALUATING POSES**

In [ ]:
poses = [' '.join(item.split(sep=' ')[:-1]) for item in poses]

In [ ]:
poses

['upright', 'upright', 'upright', 'upright']

In [ ]:
poses_dict = {}

for pose in poses:
  if pose in poses_dict.keys():
    poses_dict[pose] += 1
  else:
    poses_dict[pose] = 1

In [ ]:
poses_dict

{'upright': 4}

In [ ]:
num_poses = 3

if len(poses_dict) >= num_poses:
  max_poses = sorted(poses_dict, key=poses_dict.get, reverse=True)[:num_poses]
else:
  max_poses = sorted(poses_dict, key=poses_dict.get, reverse=True)

**EVALUATING ACTIVITIES**

In [ ]:
activities = [' '.join(item.split(sep=' ')[:-1]) for item in activities]

In [ ]:
activities_relevant = ['applauding',
                       'climbing',
                       'cooking', 
                       'cutting vegetables', 
                       'drinking', 
                       'fixing a bike', 
                       'fixing a car', 
                       'gardening', 
                       'playing guitar', 
                       'playing violin', 
                       'reading', 
                       'riding a bike', 
                       'running', 
                       'taking photos', 
                       'walking the dog']

In [ ]:
activities_dict = {}

for act in activities:
  if act in activities_relevant:
    if act in activities_dict.keys():
      activities_dict[act] += 1
    else:
      activities_dict[act] = 1

In [ ]:
activities_dict

{'applauding': 1, 'reading': 1, 'riding a bike': 1}

In [ ]:
num_activities = 3

if len(activities_dict) >= num_activities:
  max_activities = sorted(activities_dict, key=activities_dict.get, reverse=True)[:num_activities]
else:
  max_activities = sorted(activities_dict, key=activities_dict.get, reverse=True)

In [ ]:
max_activities

['reading', 'riding a bike', 'applauding']